## Carregando Libs

In [ ]:
from langchain.prompts import ChatPromptTemplate, PromptTemplate
from langchain.chains.llm import LLMChain
from langchain_ollama import ChatOllama

from langchain.chains.router import MultiPromptChain
from langchain.chains.router.llm_router import LLMRouterChain, RouterOutputParser
from langchain.chains.router.multi_prompt_prompt import MULTI_PROMPT_ROUTER_TEMPLATE

## Criando promps especificos

In [60]:
prompt_biologia =ChatPromptTemplate.from_template("""
    Você é um biólogo muito campacita. Você tem um grande conhecimento sobre os seres vivos, suas estruturas, funções
    e a interação entre diferentes organismos e seus ambientes.
    Voce é execelente em explicar conceitos de biologia de maneira clara, tanto para iniciantes quanto para estudantes
    avançados.
    Aqui está uma pergunta {input}
""")

prompt_quimica = ChatPromptTemplate.from_template("""
  Voce é um quimico muito experiente.
  Você é excelente em responder pergutnas sobre quimica de foram clara e objetiva. 
  Você tem um grande entendimento sobre reações químicas, elementos, compostos,
  e a relação entre a estrutura molecular e as propriedades dos materiais.
  Quando você não sabe a resposta para uma pergunta, você admite que não sabe.
  Aqui está uma pergunta {input}
""")

prompt_geografia =ChatPromptTemplate.from_template("""
  Você é um geógrafo muito bem infomado. 
  Você tem um vasto conhecimento sobre os processos naturais, geografia humana, clima, relevo e interações entre o ambiente e a sociedade.
  Você é habilidoso em explicar como fatores físicos e humanos afetam o mundo ao seu redor.
  Aqui está uma pergunta {input}
""")

prompt_computacao =ChatPromptTemplate.from_template("""
    Voce é um  professor ideal de Engenharia da Computação que domina as bases e as tecnologias mais recentes, traduzindo sua experiência prática 
    em didática. Ele engaja alunos, prepara-os para o mercado de trabalho com foco em habilidades interpessoais e incorpora discussões éticas sobre o impacto da tecnologia.
  Aqui está uma pergunta {input}
""")

## Criando rotas

In [47]:
prompt_infos = [
    { 
        "name":"Química",
        "description":"Ideal para responder pergunta de quimica",
        "prompt_template":prompt_quimica
    },
    { 
        "name":"Geografia",
        "description":"Ideal para responder pergunta de geografia",
        "prompt_template":prompt_geografia
    },
    { 
        "name":"Biologia",
        "description":"Ideal para responder pergunta de biologia",
        "prompt_template":prompt_biologia
    },
    { 
        "name":"Computação",
        "description":"Ideal para responder pergunta de computacao",
        "prompt_template":prompt_computacao
    }
]

In [ ]:
chat = ChatOllama(model="splitpierre/bode-alpaca-pt-br:latest")

In [49]:
chains_destiny = {}
for info in prompt_infos:
    chain = LLMChain(llm=chat, prompt=info["prompt_template"], verbose=True)
    chains_destiny[info["name"]]= chain
chains_destiny

{'Química': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    Voce é um  professor de Química ideal que desmistifica a disciplina, tornando-a visual e palpável. Ele demonstra a Química no cotidiano e na indústria, \n    enfatiza a segurança e guia os alunos no desenvolvimento do raciocínio lógico e habilidades analíticas, incentivando a curiosidade.\n  Aqui está uma pergunta {input}\n'), additional_kwargs={})]), llm=ChatOllama(model='deepseek-r1:7b'), output_parser=StrOutputParser(), llm_kwargs={}),
 'Geografia': LLMChain(verbose=True, prompt=ChatPromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, messages=[HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], input_types={}, partial_variables={}, template='\n    Voce é um professor de 

In [50]:
destinations = [f'{p["name"]}: {p["description"]}' for p in prompt_infos]

destinations_str = "\n".join(destinations)
print(destinations_str)

Química: Ideal para responder pergunta de quimica
Geografia: Ideal para responder pergunta de geografia
Biologia: Ideal para responder pergunta de biologia
Computação: Ideal para responder pergunta de computacao


In [53]:
router_template_content  = MULTI_PROMPT_ROUTER_TEMPLATE.format(destinations=destinations_str)
print(router_template_content )

Given a raw text input to a language model select the model prompt best suited for the input. You will be given the names of the available prompts and a description of what the prompt is best suited for. You may also revise the original input if you think that revising it will ultimately lead to a better response from the language model.

<< FORMATTING >>
Return a markdown code snippet with a JSON object formatted to look like:
```json
{{
    "destination": string \ name of the prompt to use or "DEFAULT"
    "next_inputs": string \ a potentially modified version of the original input
}}
```

REMEMBER: "destination" MUST be one of the candidate prompt names specified below OR it can be "DEFAULT" if the input is not well suited for any of the candidate prompts.
REMEMBER: "next_inputs" can just be the original input if you don't think any modifications are needed.

<< CANDIDATE PROMPTS >>
Química: Ideal para responder pergunta de quimica
Geografia: Ideal para responder pergunta de geograf

In [56]:
router_prompt_template  =  PromptTemplate(
    template = router_template_content ,
    input_variables=['input'],
    output_parser=RouterOutputParser()
)



router_chain = LLMRouterChain.from_llm(chat,router_prompt_template, verbose=True)

In [58]:
default_prompt = ChatPromptTemplate.from_template("{input}")
default_chain = LLMChain(llm=chat,prompt=default_prompt, verbose=True)
chain= MultiPromptChain(
    router_chain=router_chain,
    destination_chains=chains_destiny,
    default_chain=default_chain,
    verbose=True
)

C:\Users\frpbo\AppData\Local\Temp\ipykernel_14968\3373829237.py:3: LangChainDeprecationWarning: Please see migration guide here for recommended implementation: https://python.langchain.com/docs/versions/migrating_chains/multi_prompt_chain/
  chain= MultiPromptChain(


In [64]:
chain.invoke({"input": "Como posso entender o fator RH do sangue?"})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Biologia: {'input': 'Como posso entender o fator RH do sangue?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: 
    Voce é um professor de Biologia ideal que simplifica o complexo, estimula a observação e conecta a vida em sala de aula com o mundo real. 
    Ele se mantém atualizado sobre avanços científicos e transmite sua paixão pela vida, inspirando os alunos a valorizar e proteger a biodiversidade.
    Aqui está uma pergunta Como posso entender o fator RH do sangue?


> Finished chain.

> Finished chain.


{'input': 'Como posso entender o fator RH do sangue?',
 'text': '<think>\nPrimeiro, entendo que o fatorial RH do sangue é importante para compreender como os donatedores de sangue são classificados em hospitals.\n\nDescubro que hémitemo RH+ e RH- são os dois grupos básicos de classificação. O HEMTO E+ tem mais chances de ser compatível biológico, o que é importante considerar durante a doação.\n\nAcredito que os indivíduos com HEMTO E+ possuem seu HEMTO E+ reconhecido rapidamente, o que pode ajudar a evitar reações imunitárias. Por outro lado, aqueles com HEMTO- ou HEMTO B+ podem exceder os limites do sistema imunitário, o que pode levar a danos sérios.\n\nAlém disso, entendo que a compatibilidade Rh não é única e que outros fatores como a gravidade da doença e a idade também podem influenciar a segurança de um dono de sangue em relation com os recebadores.\n\nReconheço que é essencial manter atuais os conhecimentos sobre os advancementos científicos e como os padrões de doação de sang

In [65]:
chain.invoke({"input": "Para que serve a tabela periodica?"})



> Entering new MultiPromptChain chain...


> Entering new LLMRouterChain chain...

> Finished chain.
Química: {'input': 'Para que serve a tabela periodica?'}

> Entering new LLMChain chain...
Prompt after formatting:
Human: 
    Voce é um  professor de Química ideal que desmistifica a disciplina, tornando-a visual e palpável. Ele demonstra a Química no cotidiano e na indústria, 
    enfatiza a segurança e guia os alunos no desenvolvimento do raciocínio lógico e habilidades analíticas, incentivando a curiosidade.
  Aqui está uma pergunta Para que serve a tabela periodica?


> Finished chain.

> Finished chain.


{'input': 'Para que serve a tabela periodica?',
 'text': "<think>\nAh, someone is asking about the purpose of the periodic table. I should explain its importance in chemistry.\n\nFirst, it organizes elements by atomic structure and properties, making it easier to study them.\n\nIt's used for predicting chemical behavior based on position trends.\n\nIn education, it helps students learn and memorize elements efficiently.\n\nPractically, chemists use it to perform reactions and develop new substances.\n\nUnderstanding the periodic table is essential for various scientific fields.\n\nLastly, it highlights the importance of chemistry in everyday life and innovation.\n</think>\n\nA **tabela periódica** serve de fundamental工具 para o estudo e a compreensão da **química**. Ela organiza os elementos baseando-se em suas **propriedades físicas** e **químicas**, permitindo que se entenda melhor como esses elementos se relacionam entre si. Além disso, a tabela periódica é crucial para identificar p